In [27]:
import fire
import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

from tqdm.auto import tqdm
from transformers import AdamW, get_scheduler
from transformers import Trainer, TrainingArguments
import evaluate

from peft import LoraConfig, TaskType, get_peft_model

import os
from datetime import date

id2label = {0:'entailment', 1:'neutral', 2:'contradiction'}
label2id = {'entailment':0, 'neutral':1, 'contradiction':2}
num_labels = len(id2label)

model_checkpoint = "bert-base-cased"
seed: int=42
batch_size: int=128
num_train_epochs: int=3
num_proc: int=4
output_dir: str="../res"
use_peft: bool = False
do_train: bool = True
do_eval: bool=True
mini_train: bool=False

def convertlabels2ids(example):
    example['label'] = label2id[example['label']]
    return example

def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
    

def log_and_save_results(res,
    results_dir = '../../res',
    outfile_name = 'snli_model_performances.csv'
):
    outfile_path = os.path.join(results_dir, outfile_name)

    if not os.path.exists(results_dir): os.mkdir(results_dir)

    if not os.path.exists(outfile_path):
        with open(outfile_path,'a', newline='\n') as f:
            f.write("date; model_name; dataset; accuracy\n")

    today = date.today()

    for i  in res:
        model_name, dataset_str, accuracy = i
        with open(outfile_path,'a', newline='\n') as f:
            f.write(f"{today};{model_name}; {dataset_str}; {accuracy}\n")
        print(f"Accuracy of {model_name} on {dataset_str} dataset: {accuracy}")

In [3]:
checkpoint = model_checkpoint
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# BUILD DATASET
dataset = load_dataset("snli")    
def tokenize_function(examples):
    return tokenizer(examples['premise'],examples['hypothesis']) #, max_length=512, truncation=True, padding="max_length", return_tensors='pt'
tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=num_proc).filter(lambda sample: sample['label'] in list(range(num_labels)) ) 
tokenized_datasets = tokenized_datasets.rename_column('label', 'labels').remove_columns(['premise','hypothesis'])

train_dataset = tokenized_datasets["train"]
validation_dataset = tokenized_datasets["validation"]
    
# Train on smaller data for debugging purposes
mini_train = True
if mini_train:
    num_samples = int(0.1*len(train_dataset))
    print(f'only train on {num_samples} samples')
    train_dataset = train_dataset.select(range(num_samples)) 

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/550152 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/550152 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

only train on 54936 samples


In [4]:
from transformers import AutoModelForSequenceClassification
import adapters

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels)
adapters.init(model)
model.add_adapter("snli", config="seq_bn")
model.train_adapter("snli")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

summary = model.adapter_summary(as_dict=True)
print(f"trainable params: {summary[0]['#param']:,d} || all params: {summary[1]['#param']:,d} || trainable%: {summary[0]['%param']}")
num_param = summary[0]['#param']
human_format(num_param)

trainable params: 894,528 || all params: 108,310,272 || trainable%: 0.8258939650710139


'895K'

In [ ]:
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=1,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    output_dir="./adapter_training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    save_strategy = "no",
    evaluation_strategy = "epoch",
)

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,#compute_accuracy,
)

In [ ]:
trainer.train()

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['premise'],examples['hypothesis'])

test_datasets = ['snli','multi_nli','sagnikrayc/snli-bt','sagnikrayc/snli-cf-kaushik']
dataset2split = {'snli':"test", 'multi_nli':"validation_mismatched", 'sagnikrayc/snli-bt':"test", 'sagnikrayc/snli-cf-kaushik':"test"}
res = []
model_name = checkpoint.split("/")[-1]

for dataset_str in test_datasets:
    target_split = dataset2split[dataset_str] #"validation_mismatched" if dataset_str == 'multi_nli' else "test"
    dataset = load_dataset(dataset_str, split=target_split)

    if dataset_str in ['sagnikrayc/snli-bt','sagnikrayc/snli-cf-kaushik']: dataset = dataset.map(convertlabels2ids) 

    tokenized_test_dataset = dataset.map(tokenize_function, batched=True, num_proc=num_proc).filter(lambda sample: sample['label'] in list(range(num_labels)))
    
    col_names = dataset.column_names
    col_names.remove('label')
    
    tokenized_test_dataset = tokenized_test_dataset.rename_column('label', 'labels').remove_columns(col_names)
    
    results = trainer.evaluate(tokenized_test_dataset)
    res.append([model_name, dataset_str,results['eval_accuracy']])

print(res)

# Testing Main

In [1]:
import fire
import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

from tqdm.auto import tqdm
from transformers import AdamW, get_scheduler
from transformers import Trainer, TrainingArguments
import evaluate

import adapters
from adapters import AdapterTrainer

from peft import LoraConfig, TaskType, get_peft_model

import os
from datetime import datetime


id2label = {0:'entailment', 1:'neutral', 2:'contradiction'}
label2id = {'entailment':0, 'neutral':1, 'contradiction':2}
num_labels = len(id2label)

def convertlabels2ids(example):
    example['label'] = label2id[example['label']]
    return example

def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

def build_dataset(tokenizer, num_proc):
    def tokenize_function(examples):
        return tokenizer(examples['premise'],examples['hypothesis'])
    dataset = load_dataset("snli")    
    tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=num_proc).filter(lambda sample: sample['label'] in list(range(num_labels)) ) 
    tokenized_datasets = tokenized_datasets.rename_column('label', 'labels').remove_columns(['premise','hypothesis'])
    train_dataset = tokenized_datasets["train"]
    validation_dataset = tokenized_datasets["validation"]
    return train_dataset, validation_dataset

def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def evaluate_test_data(tokenizer, trainer, model_name, num_proc):
    def tokenize_function(examples):
        return tokenizer(examples['premise'],examples['hypothesis'])

    test_datasets = ['snli','multi_nli','sagnikrayc/snli-bt','sagnikrayc/snli-cf-kaushik']
    dataset2split = {'snli':"test", 'multi_nli':"validation_mismatched", 'sagnikrayc/snli-bt':"test", 'sagnikrayc/snli-cf-kaushik':"test"}
    res = []

    for dataset_str in test_datasets:
        target_split = dataset2split[dataset_str] #"validation_mismatched" if dataset_str == 'multi_nli' else "test"
        dataset = load_dataset(dataset_str, split=target_split)

        if dataset_str in ['sagnikrayc/snli-bt','sagnikrayc/snli-cf-kaushik']: dataset = dataset.map(convertlabels2ids) 

        tokenized_test_dataset = dataset.map(tokenize_function, batched=True, num_proc=num_proc).filter(lambda sample: sample['label'] in list(range(num_labels)))
        
        col_names = dataset.column_names
        col_names.remove('label')
        tokenized_test_dataset = tokenized_test_dataset.rename_column('label', 'labels').remove_columns(col_names)
        
        results = trainer.evaluate(tokenized_test_dataset)
        res.append([model_name, dataset_str,results['eval_accuracy']])
    return res

def log_and_save_results(res,
    results_dir = '../res',
    outfile_name = 'snli_model_performances.csv'
):
    outfile_path = os.path.join(results_dir, outfile_name)

    if not os.path.exists(results_dir): os.mkdir(results_dir)

    if not os.path.exists(outfile_path):
        with open(outfile_path,'a', newline='\n') as f:
            f.write("date; model_name; dataset; accuracy\n")

    today = datetime.today()

    for i  in res:
        model_name, dataset_str, accuracy = i
        with open(outfile_path,'a', newline='\n') as f:
            f.write(f"{today};{model_name}; {dataset_str}; {accuracy}\n")
        print(f"Accuracy of {model_name} on {dataset_str} dataset: {accuracy}")
    
def main(
    model_checkpoint,
    seed: int=42,
    batch_size: int=128,
    num_train_epochs: int=3,
    num_proc: int=4,
    output_dir: str="../res",
    use_adapter: bool = True,
    use_peft: bool = False,
    do_train: bool = True,
    do_eval: bool=True,
    do_log: bool=True,
    mini_train: bool=False,
    save_path: str="/nfs/turbo/umms-vgvinodv/models/finetuned-checkpoints/nlp-gen" 
):
    torch.cuda.manual_seed(seed)
    torch.manual_seed(seed)
    
    checkpoint = model_checkpoint
    metric_name = "accuracy"
    model_name = checkpoint.split("/")[-1]
    save_path = f"{save_path}/{model_name}-snli"
    
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels)
        
    # BUILD DATASET
    train_dataset, validation_dataset = build_dataset(tokenizer, num_proc)
    
    # Train on smaller data for debugging purposes
    if mini_train:
        num_samples = int(0.1*len(train_dataset))
        print(f'Only training on {num_samples} samples')
        train_dataset = train_dataset.select(range(num_samples)) 
    
    # LOAD Data_collator
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
        
    # LOAD ADAPTER
    if use_adapter:
        print("Initializing Adapters for transformer model")
        adapters.init(model)
        model.add_adapter("snli", config="seq_bn")
        model.train_adapter("snli")
        # print number of trainable parameters
        summary = model.adapter_summary(as_dict=True)
        print(f"trainable params: {summary[0]['#param']:,d} || all params: {summary[1]['#param']:,d} || trainable%: {summary[0]['%param']}")
        # edit model name
        num_param = human_format(summary[0]['#param'])
        model_name = f"ADAPTER/{model_name}-bn-adapter-{num_param}"
    
    # LOAD PEFT MODEL
    if use_peft:
        print("Loading PEFT(LORA) Model")
        lora_config = LoraConfig(
            task_type=TaskType.SEQ_CLS,
            r=16,
            lora_alpha=32,
            lora_dropout=0.05
        )
        model = get_peft_model(model, lora_config)
        # print number of trainable paramaeters
        model.print_trainable_parameters()
        # edit model name
        num_param = human_format(model.get_nb_trainable_parameters()[0])
        model_name = f"PEFT/{model_name}-lora-{num_param}"
    
    # PREPARE FOR TRAINING
    args = TrainingArguments(
        save_path,
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        save_total_limit=1,
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model=metric_name,
        overwrite_output_dir=True,
        remove_unused_columns=False,
    )
    
    if use_adapter:
        trainer = AdapterTrainer(
            model=model,
            args=args,
            train_dataset=train_dataset,
            eval_dataset=validation_dataset,
            data_collator=data_collator,
            compute_metrics=compute_metrics
        )
    else:
        trainer = Trainer(
            model,
            args,
            train_dataset=train_dataset,
            eval_dataset=validation_dataset,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )
    
    if do_train:
        trainer.train()  
    
    # EVAL PERFORMANCE    
    if do_eval:
        res = evaluate_test_data(tokenizer, trainer, model_name, num_proc)
    
        # LOG RESULT METRICS
        if do_log:
            log_and_save_results(res, results_dir = output_dir, outfile_name = 'snli_model_performances.csv')
        else:
            print(res)

In [2]:
if __name__ == "__main__":
    main(model_checkpoint="bert-base-cased", use_adapter=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

...

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Accuracy of ADAPTER/bert-base-cased on snli dataset: 0.8407980456026058
Accuracy of ADAPTER/bert-base-cased on multi_nli dataset: 0.6540886899918633
Accuracy of ADAPTER/bert-base-cased on sagnikrayc/snli-bt dataset: 0.791454223010419
Accuracy of ADAPTER/bert-base-cased on sagnikrayc/snli-cf-kaushik dataset: 0.6685


In [3]:
if __name__ == "__main__":
    main(model_checkpoint="bert-large-cased", use_adapter=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/550152 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

...

Map (num_proc=4):   0%|          | 0/18044 [00:00<?, ? examples/s]

Filter:   0%|          | 0/18044 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Accuracy of ADAPTER/bert-large-cased on snli dataset: 0.882328990228013
Accuracy of ADAPTER/bert-large-cased on multi_nli dataset: 0.7093165174938975
Accuracy of ADAPTER/bert-large-cased on sagnikrayc/snli-bt dataset: 0.827976058523609
Accuracy of ADAPTER/bert-large-cased on sagnikrayc/snli-cf-kaushik dataset: 0.719


In [4]:
if __name__ == "__main__":
    main(model_checkpoint="roberta-base", use_adapter=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/550152 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

...

Map (num_proc=4):   0%|          | 0/18044 [00:00<?, ? examples/s]

Filter:   0%|          | 0/18044 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Accuracy of ADAPTER/roberta-base on snli dataset: 0.8785627035830619
Accuracy of ADAPTER/roberta-base on multi_nli dataset: 0.7571196094385679
Accuracy of ADAPTER/roberta-base on sagnikrayc/snli-bt dataset: 0.827144757260031
Accuracy of ADAPTER/roberta-base on sagnikrayc/snli-cf-kaushik dataset: 0.727


In [5]:
if __name__ == "__main__":
    main(model_checkpoint="roberta-large", use_adapter=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/550152 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

...

Map (num_proc=4):   0%|          | 0/18044 [00:00<?, ? examples/s]

Filter:   0%|          | 0/18044 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Accuracy of ADAPTER/roberta-large on snli dataset: 0.9123574918566775
Accuracy of ADAPTER/roberta-large on multi_nli dataset: 0.8221114727420668
Accuracy of ADAPTER/roberta-large on sagnikrayc/snli-bt dataset: 0.8505874528929284
Accuracy of ADAPTER/roberta-large on sagnikrayc/snli-cf-kaushik dataset: 0.794
